In [7]:
import polars as pl

In [20]:
df = pl.read_csv("raw_data/SESNSP/IDM_NM_jul25.csv",
                 encoding="latin-1").filter(pl.col("Entidad") == "Jalisco")
df

Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
i64,i64,str,i64,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2015,14,"""Jalisco""",14001,"""Acatic""","""La vida y la Integridad corpor…","""Homicidio""","""Homicidio doloso""","""Con arma de fuego""",0,0,0,0,0,0,0,0,0,0,0,1
2015,14,"""Jalisco""",14001,"""Acatic""","""La vida y la Integridad corpor…","""Homicidio""","""Homicidio doloso""","""Con arma blanca""",0,0,0,0,0,0,0,0,0,0,0,0
2015,14,"""Jalisco""",14001,"""Acatic""","""La vida y la Integridad corpor…","""Homicidio""","""Homicidio doloso""","""Con otro elemento""",0,0,0,0,0,0,0,0,1,0,0,0
2015,14,"""Jalisco""",14001,"""Acatic""","""La vida y la Integridad corpor…","""Homicidio""","""Homicidio doloso""","""No especificado""",0,0,0,0,0,0,0,0,0,0,0,0
2015,14,"""Jalisco""",14001,"""Acatic""","""La vida y la Integridad corpor…","""Homicidio""","""Homicidio culposo""","""Con arma de fuego""",0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025,14,"""Jalisco""",14998,"""No Especificado""","""Otros bienes jurídicos afectad…","""Falsificación""","""Falsificación""","""Falsificación""",0,0,0,0,0,0,0,null,null,null,null,null
2025,14,"""Jalisco""",14998,"""No Especificado""","""Otros bienes jurídicos afectad…","""Contra el medio ambiente""","""Contra el medio ambiente""","""Contra el medio ambiente""",0,0,0,0,0,0,0,null,null,null,null,null
2025,14,"""Jalisco""",14998,"""No Especificado""","""Otros bienes jurídicos afectad…","""Delitos cometidos por servidor…","""Delitos cometidos por servidor…","""Delitos cometidos por servidor…",0,0,0,0,0,0,0,null,null,null,null,null


In [ ]:
df_cols = df.columns[:-12]
months = df.columns[-12:]
months_map = {month: i + 1 for i, month in enumerate(months)}

unpivoted_df = df.unpivot(
    index=df_cols,
    on=months,
    variable_name="Mes",
    value_name="count"
).filter(pl.col("count") > 0) # TODO: Revisar si los meses en null afectan el procesamiento

expanded_df = (
    unpivoted_df
    .with_columns(pl.struct(unpivoted_df.columns).alias("row_struct"))
    .with_columns(pl.col("row_struct").repeat_by("count").alias("expanded"))
    .drop(unpivoted_df.columns + ["count", "row_struct"])
    .explode("expanded")
    .unnest("expanded")
    .with_columns(pl.col("Mes").replace_strict(months_map).cast(pl.Int8))
    .with_columns(pl.date(pl.col("Año"), pl.col("Mes"), pl.lit(1)).alias("Fecha"))
    .drop("count", "Año", "Mes")
)

expanded_df

Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Fecha
i64,str,i64,str,str,str,str,str,date
14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Robo a casa habitación""","""Sin violencia""",2015-01-01
14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Robo de vehículo automotor""","""Robo de motocicleta Sin violen…",2015-01-01
14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",2015-01-01
14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",2015-01-01
14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",2015-01-01
…,…,…,…,…,…,…,…,…
14,"""Jalisco""",14998,"""No Especificado""","""El patrimonio""","""Fraude""","""Fraude""","""Fraude""",2024-12-01
14,"""Jalisco""",14998,"""No Especificado""","""El patrimonio""","""Fraude""","""Fraude""","""Fraude""",2024-12-01
14,"""Jalisco""",14998,"""No Especificado""","""Otros bienes jurídicos afectad…","""Falsificación""","""Falsificación""","""Falsificación""",2024-12-01


In [ ]:
df.filter(pl.col("Municipio") == "Guadalajara").filter(pl.col("Año") == 2015).filter(pl.col("Subtipo de delito") == "Otros robos")

Año,Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
i64,i64,str,i64,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
2015,14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Otros robos""","""Con violencia""",0,0,0,0,0,0,0,0,0,0,0,0
2015,14,"""Jalisco""",14001,"""Acatic""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",5,1,3,0,0,3,0,1,0,2,2,2


In [46]:
expanded_df.filter(pl.col("Municipio") == "Guadalajara").filter(pl.col("Fecha").dt.year() == 2015).filter(pl.col("Subtipo de delito") == "Otros robos")

Clave_Ent,Entidad,Cve. Municipio,Municipio,Bien jurídico afectado,Tipo de delito,Subtipo de delito,Modalidad,Fecha
i64,str,i64,str,str,str,str,str,date
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Con violencia""",2015-01-01
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Con violencia""",2015-01-01
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Con violencia""",2015-01-01
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Con violencia""",2015-01-01
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Con violencia""",2015-01-01
…,…,…,…,…,…,…,…,…
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",2015-12-01
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",2015-12-01
14,"""Jalisco""",14039,"""Guadalajara""","""El patrimonio""","""Robo""","""Otros robos""","""Sin violencia""",2015-12-01


In [48]:
df_clean = expanded_df.insert_column(0, pl.arange(0, expanded_df.height).alias("id"))
df_clean.write_parquet("data/SESNSP/SESNSP_jalisco_data.parquet")